<br><br>

## **Import necessary Python libraries and modules**

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch
#import transformers


To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 6.8 MB/s 
     |████████████████████████████████| 6.5 MB 33.5 MB/s 
     |████████████████████████████████| 67 kB 1.8 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
     |████████████████████████████████| 596 kB 37.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [3]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [4]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'distilbert-base-multilingual-cased'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 128                                                       

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Polarity_fin_mbert'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 448 kB 48.0 MB/s 


In [7]:
#pretokenizzazionen in morfemi delle koreane
from konlpy.tag import Okt
def pretok_ko(subset, indice):
  new_ko=[] 
  #count=0
  for index,row in subset.iterrows():
    if index < indice:
      new_ko.append(row['Sentence'])
    else: #quando arrivi all'indice di ko

      okt = Okt()
      li=okt.morphs(str(row['Sentence']))
      stringa=''
      for el in li:
        stringa+=str(el)+' '
      new_ko.append(stringa[:-1])
  subset['Sentence']=new_ko
  return subset
    


In [8]:
train=pd.read_csv("train_def_pol.csv")
#train=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Strategy_2\\train26_emo.csv")
train=pretok_ko(train,5947)
train

,Unnamed: 0,Sentence,Polarity_Classification
0,0,It provides a great insight into the behind th...,Neutra
1,1,A girl and her boyfriend decide to take that s...,Neutra
2,2,"The children's novel ""keys of the Kingdom Mist...",Neutra
3,3,"Monday and his creepy butler, who leave him wi...",Negativa
4,4,There he discovers a world beautifully ripe wi...,Neutra
...,...,...,...
11835,11835,보는내내 엉엉 울 엇네 요 스토리 가 쪼금 허 술 한 ? 면도 있지만 안내상 씨 연...,Positiva
11836,11836,저스틴 채트원 하고 에미 로섬 이 대체 저런 영화 왜 찍었는지 모르겠다 .,Negativa
11837,11837,부셔 버리고 싶다 ...,Negativa
11838,11838,Arthur Penhaligon 은 곧 죽 을 것 입니다 . 미스터 먼데이 와 그 ...,Neutra


In [9]:
#test=pd.read_csv("C:\\Users\\Fossati\\Desktop\\Tesi\\Dati\\Dati_FineTuningBERT\\Train_Test_emotion\\Strategy_2\\test2_red.csv")
test=pd.read_csv("test_def_pol.csv")
test=pretok_ko(test, 699)
test

,Unnamed: 0,Sentence,Polarity_Classification
0,0,This is one of the books that my dad read out ...,Positiva
1,1,Garth Nix's 'Mister Monday' was a highly cleve...,Positiva
2,2,"Includes affirmations, visualizations, and pra...",Positiva
3,3,Monday was fantasy,Neutra
4,4,Weird and dangerous creatures are hunting him ...,Neutra
...,...,...,...
1387,1387,"달리는 동안 그 는 이 것 을 작은 열쇠 라고 하는 것 을 갖게 되었고 , 그것 은...",Neutra
1388,1388,아벨라 부터 들어간 기아 순정 이긴 한데 프라이드 는 잘 모르 겠 네요 ㅋㅋ 끝물 ...,Neutra
1389,1389,이전 이나 변함 없다 .,Negativa
1390,1390,[ 3 점 ] 뭐 하자는거지 ?,Negativa


In [10]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=train['Sentence']
train_labels=train['Polarity_Classification']
test_texts=test['Sentence']
test_labels=test['Polarity_Classification']


<br><br>

## **Split the data into training and test sets**

In [11]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(11840, 11840, 1392, 1392)

Here's an example of a training label and review:

In [12]:
train_labels[0], train_texts[0]

('Neutra',
 'It provides a great insight into the behind the scenes goings-on in parliament, the rivalries and treacheries in the run up to Winston Churchill becoming Britains war time Prime Minister')

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




In [13]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [14]:
#unique_labels = set(label for label in train_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}

In [15]:
unique_labels={'Neutra', 'Positiva', 'Negativa'}
label2id={'Negativa': 2, 'Neutra': 0, 'Positiva': 1}
id2label={2:'Negativa', 0:'Neutra', 1:'Positiva'}

In [16]:
unique_labels

{'Negativa', 'Neutra', 'Positiva'}

In [17]:
label2id.keys()

dict_keys(['Negativa', 'Neutra', 'Positiva'])

In [18]:
id2label.keys()

dict_keys([2, 0, 1])

Now let's encode our texts and labels!

In [19]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=max_length)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

In [20]:
' '.join(train_encodings[29].tokens[0:100])

'[CLS] " The war ##time episodes , however , are less sat ##is ##fa ##ctory [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [21]:
' '.join(test_encodings[0].tokens[0:100])

'[CLS] This is one of the books that my dad read out lo ##ud to me at bed ##time , but when ##ever he had to put it down , I begge ##d him to read more [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [22]:
set(train_labels_encoded)

{0, 1, 2}

In [23]:
set(test_labels_encoded)

{0, 1, 2}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [24]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [25]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [26]:
' '.join(train_dataset.encodings[0].tokens[0:100])

'[CLS] It provides a great ins ##ight into the behind the scenes going ##s - on in parliament , the rival ##ries and tre ##acher ##ies in the run up to Winston Churchill becoming Britain ##s war time Prime Minister [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [27]:
' '.join(test_dataset.encodings[1].tokens[0:100])

"[CLS] Ga ##rth Ni ##x ' s ' Mister Monday ' was a highly c ##lever , creative , and enter ##taining read that had me up into the we ##e hours of the morning following Arthur ' s adventure ##s in the House [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

<br><br>

## **Load pre-trained BERT model**

Here we load a pre-trained DistilBERT model and send it to CUDA.

**Note:** If you decide to repeat fine-tuning after already running the following cells, make sure that you re-run this cell to re-load the original pre-trained model before fine-tuning again.

In [28]:
# The model_name needs to match the name used for the tokenizer above.
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['pre_classifier.weight', 'classif

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [29]:
training_args=TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=2000,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
    save_strategy='no'
)

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [30]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [31]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
)

Time to finally fine-tune! 

Be patient; if you've set everything in Colab to use GPUs, then it should only take a minute or two to run, but if you're running on CPU, it can take hours.

After every 10 steps (as we specified in the TrainingArguments object), the trainer will output the current state of the model, including the training loss, validation ("test") loss, and accuracy (from our `compute_metrics` function).

You should see the loss going down and the accuracy going up. If instead they are staying the same or oscillating, you probably need to change the fine-tuning parameters.

In [32]:
trainer.train()                                                                                                         

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11840
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2960


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Accuracy
2000,0.599000,0.334664,0.887213


***** Running Evaluation *****
  Num examples = 1392
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2960, training_loss=0.4965895266146273, metrics={'train_runtime': 1255.8933, 'train_samples_per_second': 37.71, 'train_steps_per_second': 2.357, 'total_flos': 1568441970524160.0, 'train_loss': 0.4965895266146273, 'epoch': 4.0})

In [ ]:
#tempo di training: 0 min

<br><br>

## **Save fine-tuned model**

The following cell will save the model and its configuration files to a directory in Colab. To preserve this model for future use, you should download the model to your computer.

In [33]:
trainer.save_model(cached_model_directory_name)

Saving model checkpoint to Polarity_fin_mbert
Configuration saved in Polarity_fin_mbert/config.json
Model weights saved in Polarity_fin_mbert/pytorch_model.bin


(Optional) If you've already fine-tuned and saved the model, you can reload it using the following line. You don't have to run fine-tuning every time you want to evaluate.

<br><br>

## **Evaluate fine-tuned model**

The following function of the `Trainer` object will run the built-in evaluation, including our `compute_metrics` function.

In [34]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1392
  Batch size = 16


{'epoch': 4.0,
 'eval_accuracy': 0.9145114942528736,
 'eval_loss': 0.27210935950279236,
 'eval_runtime': 11.364,
 'eval_samples_per_second': 122.492,
 'eval_steps_per_second': 7.656}

But we might want to do more fine-grained analysis of the model, so we extract the predicted labels.

In [35]:
predicted_results = trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 1392
  Batch size = 16


In [36]:
predicted_results.predictions.shape

(1392, 3)

In [37]:
predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

In [38]:
len(predicted_labels)

1392

In [39]:
print(classification_report(test_labels, 
                            predicted_labels))

              precision    recall  f1-score   support

    Negativa       0.87      0.92      0.90       460
      Neutra       0.95      0.95      0.95       467
    Positiva       0.93      0.88      0.90       465

    accuracy                           0.91      1392
   macro avg       0.92      0.91      0.91      1392
weighted avg       0.92      0.91      0.91      1392



In [40]:
#la novità è che c'è risultato generale sufficientemente buoni per tutte e 5 classi

In [41]:
#RELOAD THE MODEL

In [42]:
#valutazione per domain

In [46]:
def prediction_test(dataset):  #dataset fa riferimento al set di dati che vogliamo passare, modello dipende dal tipo di FT
#poi ci sarà anche la funzione per predire senza labels
    tok = DistilBertTokenizerFast.from_pretrained('distilbert-base-multilingual-cased')
    modello=DistilBertForSequenceClassification.from_pretrained(cached_model_directory_name)
    test_texts=dataset['Sentence']
    test_labels=dataset['Polarity_Classification']

    #unique_labels = set(label for label in test_labels)
    #label2id = {label: id for id, label in enumerate(unique_labels)}
    #id2label = {id: label for label, id in label2id.items()}
    unique_labels={'Neutra', 'Positiva', 'Negativa'}
    label2id={'Negativa': 2, 'Neutra': 0, 'Positiva': 1}
    id2label={2:'Negativa', 0:'Neutra', 1:'Positiva'}  

#train_encodings = tok(train_texts.tolist(), truncation=True, padding=True, max_length=max_length)
    test_encodings  = tok(test_texts.tolist(), truncation=True, padding=True, max_length=128)

#train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
    test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]
    

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, encodings, labels):
            self.encodings = encodings
            self.labels = labels

        def __getitem__(self, idx):
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = torch.tensor(self.labels[idx])
            return item

        def __len__(self):
            return len(self.labels)

#train_dataset = MyDataset(train_encodings, train_labels_encoded)
    test_dataset = MyDataset(test_encodings, test_labels_encoded)
    
    training_args = TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=100,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=2000,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps'     # evaluate during fine-tuning so that we can see progress
)
    
    trainer = Trainer(model=modello, args=training_args)  #basta avere il modello come parametro

    predicted_results=trainer.predict(test_dataset)
    
    predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
    predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
    predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

#len(predicted_labels)

    return print(classification_report(test_labels,predicted_labels))#,predicted_labels

In [47]:
prediction_test(test[:205])    #en

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

              precision    recall  f1-score   support

    Negativa       0.88      0.85      0.86        26
      Neutra       0.96      0.97      0.96        90
    Positiva       0.94      0.94      0.94        89

    accuracy                           0.94       205
   macro avg       0.93      0.92      0.92       205
weighted avg       0.94      0.94      0.94       205



In [48]:
#per l'inglese ci siamo, otteniamo ottimi risultati

In [49]:
prediction_test(test[205:699])   #it 

loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

              precision    recall  f1-score   support

    Negativa       0.80      0.87      0.83       124
      Neutra       0.91      0.91      0.91       204
    Positiva       0.91      0.86      0.88       166

    accuracy                           0.88       494
   macro avg       0.87      0.88      0.88       494
weighted avg       0.88      0.88      0.88       494



In [50]:
prediction_test(test[699:])  #ko   


loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/28e5b750bf4f39cc620367720e105de1501cf36ec4ca7029eba82c1d2cc47caf.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/5cbdf121f196be5f1016cb102b197b0c34009e1e658f513515f2eebef9f38093.b33e51591f94f17c238ee9b1fac75b96ff2678cbaed6e108feadb3449d18dc24
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/47087d99feeb3bc6184d7576ff089c52f7fbe

              precision    recall  f1-score   support

    Negativa       0.90      0.95      0.92       310
      Neutra       0.98      0.98      0.98       173
    Positiva       0.93      0.86      0.90       210

    accuracy                           0.93       693
   macro avg       0.94      0.93      0.93       693
weighted avg       0.93      0.93      0.93       693



In [50]:
#top mBERT